In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Grover-Algorithmus

Für dieses Qiskit in Classrooms-Modul müssen Studierende eine funktionierende Python-Umgebung mit den folgenden installierten Paketen haben:
- `qiskit` v2.1.0 oder neuer
- `qiskit-ibm-runtime` v0.40.1 oder neuer
- `qiskit-aer` v0.17.0 oder neuer
- `qiskit.visualization`
- `numpy`
- `pylatexenc`

Um die oben genannten Pakete einzurichten und zu installieren, siehe den Leitfaden [Qiskit installieren](/guides/install-qiskit).
Um Jobs auf echten Quantencomputern auszuführen, müssen Studierende ein Konto bei IBM Quantum&reg; einrichten, indem sie die Schritte im Leitfaden [Richten Sie Ihr IBM Cloud-Konto ein](/guides/cloud-setup) befolgen.

Dieses Modul wurde getestet und verwendete 12 Sekunden QPU-Zeit. Dies ist eine gutgläubige Schätzung; Ihre tatsächliche Nutzung kann variieren.

In [ ]:
# Uncomment and modify this line as needed to install dependencies
#!pip install 'qiskit>=2.1.0' 'qiskit-ibm-runtime>=0.40.1' 'qiskit-aer>=0.17.0' 'numpy' 'pylatexenc'

## Einführung

**Der Grover-Algorithmus** ist ein grundlegender Quantenalgorithmus, der das *unstrukturierte Suchproblem* adressiert: Gegeben eine Menge von $N$ Elementen und eine Möglichkeit zu überprüfen, ob ein gegebenes Element dasjenige ist, nach dem Sie suchen, wie schnell können Sie das gewünschte Element finden? Beim klassischen Computing gilt: Wenn die Daten unsortiert sind und es keine Struktur gibt, die ausgenutzt werden kann, ist der beste Ansatz, jedes Element einzeln zu überprüfen, was zu einer Abfragekomplexität von $O(N)$ führt – im Durchschnitt müssen Sie etwa die Hälfte der Elemente überprüfen, bevor Sie das Ziel finden.

![Ein Diagramm der klassischen unstrukturierten Suche.](../../../learning/images/modules/computer-science/grovers/classical-uss.avif)

Der 1996 von Lov Grover eingeführte Grover-Algorithmus zeigt, wie ein Quantencomputer dieses Problem viel effizienter lösen kann und nur $O(\sqrt{N})$ Schritte benötigt, um das markierte Element mit hoher Wahrscheinlichkeit zu finden. Dies stellt eine *quadratische Beschleunigung* gegenüber klassischen Methoden dar, was für große Datensätze signifikant ist.

Der Algorithmus arbeitet im folgenden Kontext:
- **Problemstellung:** Sie haben eine Funktion $f(x)$, die 1 zurückgibt, wenn $x$ das gewünschte Element ist, und sonst 0. Diese Funktion wird oft als *Orakel* oder *Black Box* bezeichnet, da Sie nur durch Abfragen von $f(x)$ etwas über die Daten erfahren können.
- **Nutzen von Quanten:** Während klassische Algorithmen für dieses Problem im Durchschnitt $N/2$ Abfragen erfordern, kann der Grover-Algorithmus die Lösung in ungefähr $\pi\sqrt{N}/4$ Abfragen finden, was für große $N$ viel schneller ist.
- **Wie es funktioniert (auf hoher Ebene):**
  - Der Quantencomputer erzeugt zunächst eine *Überlagerung* aller möglichen Zustände, die alle möglichen Elemente gleichzeitig darstellt.
  - Dann wendet er wiederholt eine Sequenz von Quantenoperationen (die Grover-Iteration) an, die die Wahrscheinlichkeit der richtigen Antwort amplifiziert und die anderen vermindert.
  - Nach genügend Iterationen liefert die Messung des Quantenzustands mit hoher Wahrscheinlichkeit die richtige Antwort.

Hier ist ein sehr einfaches Diagramm des Grover-Algorithmus, das viele Nuancen überspringt. Für ein detaillierteres Diagramm siehe [dieses Paper](https://arxiv.org/pdf/2211.04543).

![Ein Diagramm auf hoher Ebene der Schritte zur Implementierung des Grover-Algorithmus.](../../../learning/images/modules/computer-science/grovers/quantum-uss2.avif)

Einige Dinge, die über den Grover-Algorithmus zu beachten sind:
- Er ist optimal für unstrukturierte Suche: Kein Quantenalgorithmus kann das Problem mit weniger als $O(\sqrt{N})$ Abfragen lösen.
- Er bietet nur eine quadratische, keine exponentielle Beschleunigung – im Gegensatz zu einigen anderen Quantenalgorithmen (z. B. Shors Algorithmus zum Faktorisieren).
- Er hat praktische Implikationen, wie z. B. die potenzielle Beschleunigung von Brute-Force-Angriffen auf kryptografische Systeme, obwohl die Beschleunigung nicht ausreicht, um die meiste moderne Verschlüsselung allein zu brechen.

Für Studierende im Grundstudium, die mit grundlegenden Computing-Konzepten und Abfragemodellen vertraut sind, bietet der Grover-Algorithmus eine klare Illustration, wie Quantencomputing klassische Ansätze für bestimmte Probleme übertreffen kann, auch wenn die Verbesserung "nur" quadratisch ist. Er dient auch als Tor zum Verständnis fortgeschrittenerer Quantenalgorithmen und des breiteren Potenzials des Quantencomputings.

Amplituden-Amplifikation ist ein Quantenalgorithmus oder eine Unterroutine für allgemeine Zwecke, die verwendet werden kann, um eine quadratische Beschleunigung gegenüber einer Handvoll klassischer Algorithmen zu erzielen. [Der Grover-Algorithmus](https://arxiv.org/abs/quant-ph/9605043) war der erste, der diese Beschleunigung bei unstrukturierten Suchproblemen demonstrierte. Die Formulierung eines Grover-Suchproblems erfordert eine Orakelfunktion, die einen oder mehrere rechnerische Basiszustände als die Zustände markiert, an deren Auffinden wir interessiert sind, und einen Amplifikationsschaltkreis, der die Amplitude markierter Zustände erhöht und folglich die verbleibenden Zustände unterdrückt.

Hier demonstrieren wir, wie man Grover-Orakel konstruiert und den `GroverOperator` aus der Qiskit-Schaltkreisbibliothek verwendet, um eine Grover-Suchinstanz einfach einzurichten. Das Runtime-`Sampler`-Primitiv ermöglicht eine nahtlose Ausführung von Grover-Schaltkreisen.

## Mathematik

Angenommen, es existiert eine Funktion $f$, die Binärstrings auf eine einzelne binäre Variable abbildet, d. h.
$$
f: \Sigma^n \rightarrow \Sigma
$$
Ein Beispiel, definiert auf $\Sigma^6$, ist
$$
f(x)= \begin{cases} 1 \qquad \text{wenn }x={010101}\\
0 \qquad \text{sonst }
\end{cases}
$$
Ein weiteres Beispiel, definiert auf $\Sigma^{2n}$, ist
$$
f(x)= \begin{cases} 1 \qquad \text{wenn gleiche Anzahl von 1en und 0en im String}\\
0 \qquad \text{sonst }
\end{cases}
$$
Sie sind beauftragt, Quantenzustände zu finden, die denjenigen Argumenten $x$ von $f(x)$ entsprechen, die auf 1 abgebildet werden. Mit anderen Worten, finden Sie alle ${x_1}\in \Sigma^n$ so, dass $f(x_1)=1$ (oder wenn es keine Lösung gibt, melden Sie das). Wir würden Nicht-Lösungen als $x_0$ bezeichnen. Natürlich werden wir dies auf einem Quantencomputer tun, unter Verwendung von Quantenzuständen, daher ist es nützlich, diese Binärstrings als Zustände auszudrücken:
$$
{|x_1\rangle} \in |\Sigma^n\rangle
$$
Mit der Quantenzustands-(Dirac-)Notation suchen wir nach einem oder mehreren speziellen Zuständen ${|x_1\rangle}$ in einer Menge von $N=2^n$ möglichen Zuständen, wobei $n$ die Anzahl der Qubits ist, und mit Nicht-Lösungen bezeichnet als ${|x_0\rangle}.$

Wir können uns die Funktion $f$ als von einem Orakel bereitgestellt vorstellen: eine Black Box, die wir abfragen können, um ihre Wirkung auf einen Zustand $|x\rangle$ zu bestimmen. In der Praxis werden wir die Funktion oft kennen, aber sie kann sehr kompliziert zu implementieren sein, was bedeutet, dass die Reduzierung der Anzahl von Abfragen oder Anwendungen von $f$ wichtig sein könnte. Alternativ können wir uns ein Paradigma vorstellen, in dem eine Person ein von einer anderen Person kontrolliertes Orakel abfragt, sodass wir die Orakelfunktion nicht kennen, sondern nur ihre Wirkung auf bestimmte Zustände durch Abfragen kennen.

Dies ist ein "unstrukturiertes Suchproblem", insofern es nichts Besonderes an $f$ gibt, das uns bei unserer Suche hilft. Die Ausgaben sind weder sortiert noch sind Lösungen bekanntermaßen gruppiert, und so weiter. Betrachten Sie als Analogie alte Papier-Telefonbücher. Diese unstrukturierte Suche wäre wie das Durchsuchen, um eine bestimmte __Nummer__ zu finden, und nicht wie das Durchsuchen einer alphabetisierten Namensliste.

Im Fall, dass eine einzelne Lösung gesucht wird, erfordert dies klassisch eine Anzahl von Abfragen, die linear in $N$ ist. Klar, Sie könnten eine Lösung beim ersten Versuch finden, oder Sie könnten in den ersten $N-1$ Vermutungen keine Lösungen finden, sodass Sie die $N$-te Eingabe abfragen müssen, um zu sehen, ob es überhaupt eine Lösung gibt. Da die Funktionen keine ausbeutbare Struktur haben, benötigen Sie im Durchschnitt $N/2$ Vermutungen. Der Grover-Algorithmus erfordert eine Anzahl von Abfragen oder Berechnungen von $f$, die wie $\sqrt{N}$ skaliert.

## Skizze der Schaltkreise im Grover-Algorithmus

Eine vollständige mathematische Durcharbeitung des Grover-Algorithmus findet sich beispielsweise in [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms), einem Kurs von John Watrous auf IBM Quantum Learning. Eine verdichtete Behandlung ist in einem Anhang am Ende dieses Moduls enthalten. Aber vorerst werden wir nur die Gesamtstruktur des Quantenschaltkreises überprüfen, der den Grover-Algorithmus implementiert.

Der Grover-Algorithmus kann in die folgenden Stufen unterteilt werden:
* Vorbereitung einer anfänglichen Überlagerung (Anwendung von Hadamard-Gates auf alle Qubits)
* "Markierung" des Zielzustands (der Zielzustände) mit einem Phasen-Flip
* Eine "Diffusions"-Stufe, in der Hadamard-Gates und ein Phasen-Flip auf __alle__ Qubits angewendet werden.
* Mögliche Wiederholungen der Markierungs- und Diffusionsstufen, um die Wahrscheinlichkeit zu maximieren, den Zielzustand zu messen
* Messung

![Ein Quantenschaltkreisdiagramm, das die grundlegende Einrichtung des Grover-Algorithmus zeigt. Dieses Beispiel verwendet vier Qubits.](../../../learning/images/modules/computer-science/grovers/grover-circuit-diagram-2.avif)
Oft werden das Markierungsgate $Z_f$ und die Diffusionsschichten bestehend aus $H,$ $Z_{\text{OR}},$ und $H$ gemeinsam als der "Grover-Operator" bezeichnet. In diesem Diagramm wird nur eine einzige Wiederholung des Grover-Operators gezeigt.

Hadamard-Gates $H$ sind bekannt und werden im Quantencomputing weit verbreitet eingesetzt. Das Hadamard-Gate erzeugt Überlagerungszustände. Konkret ist es definiert durch
$$
H|0\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle+|1\rangle\right)\\
H|1\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle-|1\rangle\right)
$$
Seine Operation auf jeden anderen Zustand ist durch Linearität definiert.
Insbesondere ermöglicht uns eine Schicht von Hadamard-Gates, vom Anfangszustand mit allen Qubits in $|0\rangle$ (bezeichnet als $|0\rangle^{\otimes n}$) zu einem Zustand zu gelangen, in dem jedes Qubit eine gewisse Wahrscheinlichkeit hat, entweder in $|0\rangle$ oder $|1\rangle$ gemessen zu werden; dies lässt uns den Raum aller möglichen Zustände anders untersuchen als im klassischen Computing.

Eine wichtige Folgerungseigenschaft des Hadamard-Gates ist, dass eine zweite Anwendung solche Überlagerungszustände rückgängig machen kann:
$$
H\frac{1}{\sqrt{2}}\left(|0\rangle+|1\rangle\right)=|0\rangle\\
H\frac{1}{\sqrt{2}}\left(|0\rangle-|1\rangle\right)=|1\rangle
$$

Dies wird gleich wichtig sein.

### Überprüfen Sie Ihr Verständnis

Lesen Sie die Frage unten, denken Sie über Ihre Antwort nach, dann klicken Sie auf das Dreieck, um die Lösung zu enthüllen.

<details>
<summary>

Demonstrieren Sie ausgehend von der Definition des Hadamard-Gates, dass eine zweite Anwendung des Hadamard-Gates solche Überlagerungen wie oben behauptet rückgängig macht.

</summary>

__Antwort:__

Wenn wir X auf den $|+\rangle$-Zustand anwenden, erhalten wir den Wert +1 und auf den Zustand $|-\rangle$ erhalten wir -1, also wenn wir eine 50-50-Verteilung hätten, würden wir einen Erwartungswert von 0 erhalten.

</details>

Das $Z_\text{OR}$-Gate ist weniger verbreitet und ist definiert gemäß
$$
\text{Z}_\text{OR}|x\rangle = \begin{cases}
|x\rangle & \text{wenn } x = 0^n \\
    -|x\rangle  & \text{wenn } x \neq 0^n
\end{cases}
\qquad \forall x \in \Sigma^n
$$

Schließlich ist das $Z_f$-Gate definiert durch
$$
Z_f:|x\rangle \rightarrow (-1)^{f(x)}|x\rangle \qquad \forall x \in \Sigma^n
$$

Beachten Sie, dass die Wirkung davon ist, dass $Z_f$ das Vorzeichen an einem Zielzustand umdreht, für den $f(x) = 1$ gilt, und andere Zustände unbeeinflusst lässt.

Auf einer sehr hohen, abstrakten Ebene können Sie über die Schritte im Schaltkreis auf folgende Weise nachdenken:
* Erste Hadamard-Schicht: versetzt die Qubits in eine Überlagerung aller möglichen Zustände.
* $Z_f$: markiert den Zielzustand (die Zielzustände), indem ein "-"-Vorzeichen davor gesetzt wird. Dies ändert nicht sofort die Messwahrscheinlichkeiten, aber es ändert, wie sich der Zielzustand in nachfolgenden Schritten verhalten wird.
* Eine weitere Hadamard-Schicht: Das im vorherigen Schritt eingeführte "-"-Vorzeichen wird das relative Vorzeichen zwischen einigen Termen ändern. Da Hadamard-Gates eine Mischung von rechnerischen Zuständen $(|0\rangle+|1\rangle)/\sqrt{2}$ in einen rechnerischen Zustand, $|0\rangle,$ umwandeln, und sie $(|0\rangle-|1\rangle)/\sqrt{2}$ in $|1\rangle$ umwandeln, kann dieser relative Vorzeichenunterschied nun beginnen, eine Rolle dabei zu spielen, welche Zustände gemessen werden.
* Eine letzte Schicht von Hadamard-Gates wird angewendet, und dann werden Messungen vorgenommen.
Wir werden im nächsten Abschnitt detaillierter sehen, wie dies funktioniert.

### Beispiel

Um besser zu verstehen, wie der Grover-Algorithmus funktioniert, arbeiten wir ein kleines Zwei-Qubit-Beispiel durch. Dies kann als optional für diejenigen betrachtet werden, die sich nicht auf Quantenmechanik und Dirac-Notation konzentrieren. Aber für diejenigen, die hoffen, wesentlich mit Quantencomputern zu arbeiten, ist dies sehr zu empfehlen.

Hier ist das Schaltkreisdiagramm mit den an verschiedenen Positionen beschrifteten Quantenzuständen. Beachten Sie, dass es mit nur zwei Qubits nur vier mögliche Zustände gibt, die unter allen Umständen gemessen werden könnten: $|00\rangle$, $|01\rangle$, $|10\rangle$ und $|11\rangle$.

![Ein Diagramm eines Quantenschaltkreises, der den Grover-Algorithmus auf zwei Qubits implementiert.](../../../learning/images/modules/computer-science/grovers/grover-circuit-diagram-2-q-ex.avif)

Nehmen wir an, dass das Orakel ($Z_f$, uns unbekannt) den Zustand $|01\rangle$ markiert. Wir werden die Aktionen jeder Menge von Quantengattern durcharbeiten, einschließlich des Orakels, und sehen, welche Verteilung möglicher Zustände zum Zeitpunkt der Messung herauskommt.
Zu Beginn haben wir
$$
|\psi_0\rangle = |00\rangle
$$
Mit der Definition der Hadamard-Gates haben wir
$$
|\psi_1\rangle = \frac{1}{2}\left(|0\rangle+|1\rangle\right)\left(|0\rangle+|1\rangle\right)=\frac{1}{2}\left(|00\rangle+|01\rangle+|10\rangle+|11\rangle\right)
$$
Jetzt markiert das Orakel den Zielzustand:
$$
|\psi_2\rangle = \frac{1}{2}\left(|00\rangle-|01\rangle+|10\rangle+|11\rangle\right)
$$
Beachten Sie, dass in diesem Zustand alle vier möglichen Ergebnisse die gleiche Wahrscheinlichkeit haben, gemessen zu werden. Sie haben alle ein Gewicht der Größe $1/2,$ was bedeutet, dass sie jeweils eine $|1/2|^2=1/4$ Chance haben, gemessen zu werden. Während also der Zustand $|01\rangle$ durch die "-"-Phase markiert ist, hat dies noch nicht zu einer erhöhten Wahrscheinlichkeit geführt, diesen Zustand zu messen. Wir fahren fort, indem wir die nächste Schicht von Hadamard-Gates anwenden.
$$
\begin{aligned}
|\psi_3\rangle = &\frac{1}{4}\left(|00\rangle+|01\rangle+|10\rangle+|11\rangle\right)\\
-&\frac{1}{4}\left(|00\rangle-|01\rangle+|10\rangle-|11\rangle\right)\\
+&\frac{1}{4}\left(|00\rangle+|01\rangle-|10\rangle-|11\rangle\right)\\
+&\frac{1}{4}\left(|00\rangle-|01\rangle-|10\rangle+|11\rangle\right)
\end{aligned}
$$
Wenn wir gleichartige Terme kombinieren, finden wir
$$
|\psi_3\rangle = \frac{1}{2}\left(|00\rangle+|01\rangle-|10\rangle+|11\rangle\right)
$$
Jetzt dreht $Z_{\text{OR}}$ das Vorzeichen an allen Zuständen außer $|00\rangle$ um:
$$
|\psi_4\rangle = \frac{1}{2}\left(|00\rangle-|01\rangle+|10\rangle-|11\rangle\right)
$$
Und schließlich wenden wir die letzte Schicht von Hadamard-Gates an:
$$
\begin{aligned}
|\psi_5\rangle =&\frac{1}{4}\left(|00\rangle+|01\rangle+|10\rangle+|11\rangle\right)\\
-&\frac{1}{4}\left(|00\rangle-|01\rangle+|10\rangle-|11\rangle\right)\\
+&\frac{1}{4}\left(|00\rangle+|01\rangle-|10\rangle-|11\rangle\right)\\
-&\frac{1}{4}\left(|00\rangle-|01\rangle-|10\rangle+|11\rangle\right)
\end{aligned}
$$
Es lohnt sich, die Kombination dieser Terme durchzuarbeiten, um sich selbst davon zu überzeugen, dass das Ergebnis tatsächlich ist:
$$
|\psi_5\rangle =|01\rangle
$$
Das heißt, die Wahrscheinlichkeit, $|01\rangle$ zu messen, beträgt 100% (in Abwesenheit von Rauschen und Fehlern) und die Wahrscheinlichkeit, irgendeinen anderen Zustand zu messen, ist null.

Dieses Zwei-Qubit-Beispiel war ein besonders sauberer Fall; der Grover-Algorithmus wird nicht immer so funktionieren, dass eine 100%-ige Chance ergibt, den Zielzustand zu messen. Vielmehr wird er die Wahrscheinlichkeit amplifizieren, den Zielzustand zu messen. Außerdem muss der Grover-Operator möglicherweise mehr als einmal wiederholt werden.

Im nächsten Abschnitt werden wir diesen Algorithmus mit echten IBM&reg; Quantencomputern in die Praxis umsetzen.

## Offensichtlicher Vorbehalt

Um den Grover-Algorithmus anzuwenden, mussten wir den Grover-Operator konstruieren, was bedeutet, dass wir in der Lage sein müssen, die Phase an Zuständen umzudrehen, die unsere Lösungskriterien erfüllen. Dies wirft die Frage auf: Wenn wir unsere Lösungsmenge so gut kennen, dass wir einen Quantenschaltkreis entwerfen können, um jeden zu beschriften, wonach suchen wir dann?! Die Antwort ist dreifach, und wir werden dies im gesamten Tutorial noch einmal aufgreifen:

**(1) Diese Arten von Abfragealgorithmen beinhalten oft zwei Parteien**: eine, die das Orakel hat, das die Lösungskriterien festlegt, und eine andere, die versucht, einen Lösungszustand zu erraten/zu finden. Die Tatsache, dass eine Person das Orakel bauen kann, negiert nicht die Notwendigkeit für die Suche.

**(2) Es gibt Probleme, für die es einfacher ist, ein Lösungskriterium anzugeben, als es ist, die Lösung zu finden.** Das bekannteste Beispiel hierfür ist wahrscheinlich die Identifizierung von Primfaktoren großer Zahlen. Der Grover-Algorithmus ist nicht besonders effektiv beim Lösen dieses spezifischen Problems; wir würden den Shor-Algorithmus für die Primfaktorisierung verwenden. Dies ist nur ein Beispiel, um darauf hinzuweisen, dass das Kennen des Kriteriums für das Verhalten eines Zustands nicht immer dasselbe ist wie das Kennen des Zustands.

**(3) Der Grover-Algorithmus gibt nicht nur klassische Daten zurück.** Wenn wir eine Messung des Endzustands nach $t$ Wiederholungen des Grover-Operators vornehmen, erhalten wir wahrscheinlich klassische Informationen, die den Lösungszustand identifizieren. Aber was ist, wenn wir keine klassischen Informationen wollen; was ist, wenn wir einen auf einem Quantencomputer vorbereiteten Lösungszustand für die weitere Verwendung in einem anderen Algorithmus wollen? Der Grover-Algorithmus erzeugt tatsächlich einen Quantenzustand mit stark gewichteten Lösungen. Sie können also erwarten, den Grover-Algorithmus als Unterroutine in komplizierteren Quantenalgorithmen zu finden.

Mit diesen im Hinterkopf arbeiten wir mehrere Beispiele durch. Wir beginnen mit einem Beispiel, in dem der Lösungszustand klar angegeben ist, damit wir der Logik des Algorithmus folgen können, und wir werden zu Beispielen übergehen, bei denen die Nützlichkeit des Grover-Algorithmus deutlicher wird.

## Allgemeine Importe und Ansatz

Wir beginnen mit dem Import mehrerer notwendiger Pakete.

In [11]:
# Built-in modules
import math

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import grover_operator, MCMTGate, ZGate
from qiskit.visualization import plot_distribution
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In diesem und anderen Tutorials werden wir ein Framework für Quantencomputing verwenden, das als "Qiskit-Muster" bekannt ist und Arbeitsabläufe in die folgenden Schritte unterteilt:

- Schritt 1: Klassische Eingaben auf ein Quantenproblem abbilden
- Schritt 2: Problem für die Quantenausführung optimieren
- Schritt 3: Ausführung mit Qiskit Runtime Primitives
- Schritt 4: Nachbearbeitung und klassische Analyse

Wir werden diesen Schritten im Allgemeinen folgen, obwohl wir sie möglicherweise nicht immer explizit kennzeichnen.

## Aktivität 1: Einen einzelnen gegebenen Zielzustand finden

## Schritt 1: Klassische Eingaben auf ein Quantenproblem abbilden

Wir benötigen das Phasenabfrage-Gate, um eine Gesamtphase (-1) auf Lösungszustände zu setzen und die Nicht-Lösungszustände unbeeinflusst zu lassen. Eine andere Art, dies zu sagen, ist, dass der Grover-Algorithmus ein Orakel erfordert, das einen oder mehrere markierte rechnerische Basiszustände spezifiziert, wobei "markiert" einen Zustand mit einer Phase von -1 bedeutet. Dies geschieht unter Verwendung eines Controlled-Z-Gates oder seiner mehrfach kontrollierten Verallgemeinerung über $N$ Qubits. Um zu sehen, wie dies funktioniert, betrachten Sie ein spezifisches Beispiel eines Bitstrings `{110}`. Wir möchten einen Schaltkreis, der auf einen Zustand $|\psi\rangle = |q_2,q_1,q_0\rangle$ wirkt und eine Phase anwendet, wenn $|\psi\rangle = |011\rangle$ (wo wir die Reihenfolge des Binärstrings umgedreht haben, wegen der Notation in Qiskit, die das am wenigsten signifikante (oft 0) Qubit auf die rechte Seite setzt).

Somit wollen wir einen Schaltkreis $Z_f$, der erreicht

$$
Z_f|\psi\rangle = \begin{cases} -|\psi\rangle \qquad \text{wenn} \qquad |\psi\rangle = |011\rangle \\ |\psi\rangle \qquad \text{wenn} \qquad |\psi\rangle \neq |011\rangle\end{cases}
$$
Wir können das Multiple Control Multiple Target Gate (`MCMTGate`) verwenden, um ein Z-Gate anzuwenden, das von allen Qubits kontrolliert wird (die Phase umdrehen, wenn alle Qubits im $|1\rangle$-Zustand sind). Natürlich können einige der Qubits in unserem gewünschten Zustand $|0\rangle$ sein. Daher müssen wir für diese Qubits zuerst ein X-Gate anwenden, dann das mehrfach kontrollierte Z-Gate ausführen, dann ein weiteres X-Gate anwenden, um unsere Änderung rückgängig zu machen. Das `MCMTGate` sieht so aus:

In [23]:
mcmt_ex = QuantumCircuit(3)
mcmt_ex.compose(MCMTGate(ZGate(), 3 - 1, 1), inplace=True)
mcmt_ex.draw(output="mpl", style="iqp")

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/66aeceae-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zelle](../../../learning/images/modules/computer-science/grovers/extracted-outputs/66aeceae-0.avif)

Beachten Sie, dass viele Qubits am Kontrollprozess beteiligt sein können (hier sind es drei Qubits), aber kein einzelnes Qubit wird als Ziel bezeichnet. Dies liegt daran, dass der gesamte Zustand ein Gesamt-"-"-Vorzeichen erhält (Phasenumkehr); das Gate wirkt auf alle Qubits gleichwertig. Dies unterscheidet sich von vielen anderen Mehrqubit-Gates, wie dem `CX`-Gate, das ein einzelnes Kontrollqubit und ein einzelnes Zielqubit hat.

Im folgenden Code definieren wir ein Phasenabfrage-Gate (oder Orakel), das genau das tut, was wir oben beschrieben haben: markiert einen oder mehrere Eingabebasiszustände, die durch ihre Bitstring-Darstellung definiert sind. Das MCMT-Gate wird verwendet, um das mehrfach kontrollierte Z-Gate zu implementieren.

In [12]:
def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bitstring to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bitstring
        zero_inds = [
            ind for ind in range(num_qubits) if rev_target.startswith("0", ind)
        ]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bitstring has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMTGate(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

Jetzt wählen wir einen bestimmten "markierten" Zustand als unser Ziel und wenden die gerade definierte Funktion an. Schauen wir uns an, welche Art von Schaltkreis sie erstellt hat.

In [13]:
marked_states = ["1110"]
oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/6cb8ce21-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zelle](../../../learning/images/modules/computer-science/grovers/extracted-outputs/6cb8ce21-0.avif)

Wenn die Qubits 1-3 im $|1\rangle$-Zustand sind und Qubit 0 anfänglich im $|0\rangle$-Zustand ist, wird das erste X-Gate Qubit 0 auf $|1\rangle$ umdrehen und alle Qubits werden in $|1\rangle$ sein. Dies bedeutet, dass das MCMT-Gate eine Gesamt-Vorzeichenänderung oder Phasenumkehr anwenden wird, wie gewünscht. Für jeden anderen Fall sind entweder die Qubits 1-3 im $|0\rangle$-Zustand, oder Qubit 0 wird auf den $|0\rangle$-Zustand umgedreht, und die Phasenumkehr wird nicht angewendet. Wir sehen, dass dieser Schaltkreis tatsächlich unseren gewünschten Zustand $|0111\rangle,$ oder den Bitstring `{1110}` markiert.
Der vollständige Grover-Operator besteht aus dem Phasenabfrage-Gate (Orakel), Hadamard-Schichten und dem $Z_\text{OR}$-Operator. Wir können den eingebauten `grover_operator` verwenden, um diesen aus dem oben definierten Orakel zu konstruieren.

In [14]:
grover_op = grover_operator(oracle)
grover_op.decompose(reps=0).draw(output="mpl", style="iqp")

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/9426f7a5-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zelle](../../../learning/images/modules/computer-science/grovers/extracted-outputs/9426f7a5-0.avif)

Wie wir oben argumentiert haben, müssen wir möglicherweise den Grover-Operator mehrmals anwenden. Die optimale Anzahl von Iterationen, $t,$ um die Amplitude des Zielzustands in Abwesenheit von Rauschen zu maximieren, kann aus diesem Ausdruck erhalten werden:
$$
(2t+1)\theta = (2t+1)\sin^{-1}\left( \sqrt{\frac{|A_1|}{N}}\right) \approx (2t+1)\sqrt{\frac{|A_1|}{N}} \approx \frac{\pi}{2}\\
t\approx \frac{\pi}{4} \sqrt{\frac{N}{|A_1|}}-\frac{1}{2}
$$
Hier ist $A_1$ die Anzahl der Lösungen oder Zielzustände. Bei modernen verrauschten Quantencomputern könnte die experimentell optimale Anzahl von Iterationen anders sein - aber hier berechnen und verwenden wir diese theoretische, optimale Zahl unter Verwendung von $A_1=1$.

In [ ]:
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)
print(optimal_num_iterations)

3


Let us now construct a circuit that includes the initial Hadamard gates to create a superposition of all possible states, and apply the Grover operator the optimal number of times.

In [16]:
qc = QuantumCircuit(grover_op.num_qubits)
# Create even superposition of all basis states
qc.h(range(grover_op.num_qubits))
# Apply Grover operator the optimal number of times
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure all qubits
qc.measure_all()
qc.draw(output="mpl", style="iqp")

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/63006e25-0.avif" alt="Output of the previous code cell" />

Konstruieren wir nun einen Schaltkreis, der die anfänglichen Hadamard-Gates enthält, um eine Überlagerung aller möglichen Zustände zu erzeugen, und wenden wir den Grover-Operator die optimale Anzahl von Malen an.

In [ ]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
from qiskit_ibm_runtime import QiskitRuntimeService

# Syntax for first saving your token.  Delete these lines after saving your credentials.
# QiskitRuntimeService.save_account(channel='ibm_quantum_platform', instance = '<YOUR_IBM_INSTANCE_CRN>', token='<YOUR_API_KEY>', overwrite=True, set_as_default=True)
# service = QiskitRuntimeService(channel='ibm_quantum_platform')

# Load saved credentials
service = QiskitRuntimeService()

backend = service.least_busy(operational=True, simulator=False)
backend.name

qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-08-08 14:14:19,931: Default instance not set. Searching all available instances.


'ibm_brisbane'

![Ausgabe der vorherigen Code-Zelle](../../../learning/images/modules/computer-science/grovers/extracted-outputs/63006e25-0.avif)

Wir haben unseren Grover-Schaltkreis konstruiert!

## Schritt 2: Problem für die Ausführung auf Quanten-Hardware optimieren
Wir haben unseren abstrakten Quantenschaltkreis definiert, aber wir müssen ihn in Bezug auf Gates umschreiben, die nativ für den Quantencomputer sind, den wir tatsächlich verwenden möchten. Wir müssen auch angeben, welche Qubits auf dem Quantencomputer verwendet werden sollen. Aus diesen und anderen Gründen müssen wir nun unseren Schaltkreis transpilieren. Geben wir zunächst den Quantencomputer an, den wir verwenden möchten.

Unten ist Code zum Speichern Ihrer Anmeldeinformationen bei der ersten Verwendung. Stellen Sie sicher, dass Sie diese Informationen aus dem Notebook löschen, nachdem Sie sie in Ihre Umgebung gespeichert haben, damit Ihre Anmeldeinformationen nicht versehentlich geteilt werden, wenn Sie das Notebook teilen. Siehe [Richten Sie Ihr IBM Cloud-Konto ein](/guides/initialize-account) und [Initialisieren Sie den Service in einer nicht vertrauenswürdigen Umgebung](/guides/cloud-setup-untrusted) für weitere Anleitungen.

In [171]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

circuit_isa = pm.run(qc)
# The transpiled circuit will be very large. Only draw it if you are really curious.
# circuit_isa.draw(output="mpl", idle_wires=False, style="iqp")

It is worth noting at this time that the depth of the transpiled quantum circuit is substantial.

In [172]:
print("The total depth is ", circuit_isa.depth())
print(
    "The depth of two-qubit gates is ",
    circuit_isa.depth(lambda instruction: instruction.operation.num_qubits == 2),
)

The total depth is  439
The depth of two-qubit gates is  113


These are actually quite large numbers, even for this simple case. Since all quantum gates (and especially two-qubit gates) experience errors and are subject to noise, a series of over 100 two-qubit gates would result in nothing but noise if the qubits were not extremely high-performing. Let's see how these perform.

## Execute using Qiskit primitives

We want to make many measurements and see which state is the most likely. Such an amplitude amplification is a sampling problem that is suitable for execution with the `Sampler` Qiskit Runtime primitive.

Note that the `run()` method of Qiskit Runtime SamplerV2 takes an iterable of primitive unified blocks (PUBs). For Sampler, each PUB is an iterable in the format (circuit, parameter_values). However, at a minimum, it takes a list of quantum circuit(s).

In [96]:
# To run on a real quantum computer (this was tested on a Heron r2 processor and used 4 sec. of QPU time)

from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(mode=backend)
sampler.options.default_shots = 10_000
result = sampler.run([circuit_isa]).result()
dist = result[0].data.meas.get_counts()

Jetzt verwenden wir einen Preset-Pass-Manager, um unseren Quantenschaltkreis für das ausgewählte Backend zu optimieren.

In [ ]:
# To run on local simulator:
# from qiskit.primitives import StatevectorSampler as Sampler
# sampler = Sampler()
# result = sampler.run([qc]).result()
# dist = result[0].data.meas.get_counts()

Es ist an dieser Stelle erwähnenswert, dass die Tiefe des transpilierten Quantenschaltkreises erheblich ist.

In [97]:
plot_distribution(dist)

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/96a9107e-0.avif" alt="Output of the previous code cell" />

We see that Grover's algorithm returned the desired state with the highest probability by far, at least an order of magnitude higher than other options. In the next activity, we will use the algorithm in a way that is more consistent with the two-party workflow of a query algorithm.

#### Check your understanding
Read the questions below, think about your answer, then click the triangle to reveal the solution.

<details>
<summary>

We just searched for a single solution in a set of $2^4=16$ possible states. We determined the optimal number of repetitions of the Grover operator to be $t=3$. Would this optimal number have increased or decreased if we had searched for (a) any of several solutions, or (b) a single solution in a space of more possible states?

</summary>

__Answer:__

Recall that as long as the number of solutions is small compared to the entire space of solutions, we can expand the sine function around small angles and use
$$
(2t+1)\theta = (2t+1) \sin^{-1}{\sqrt{\frac{|\mathcal{A}_1|}{N}}}\approx (2t+1) \sqrt{\frac{|\mathcal{A}_1|}{N}} \approx \pi/2\\

t \approx \frac{\pi}{4}\sqrt{\frac{N}{|\mathcal{A}_1|}}-\frac{1}{2}
$$

(a) We see from the above expression that increasing the number of solution states would decrease the number of iterations. Provided that the fraction $\frac{|\mathcal{A}_1|}{N}$ is still small, we can describe how $t$ would decrease: $t~\frac{1}{\sqrt{|\mathcal{A}_1|}}.$

(b) As the space of possible solutions ($N$) increases, the number of required iterations increases, but only like $t~\sqrt{N}$.

</details>

<details>
<summary>

Suppose we could increase the size of the target bitstring to be arbitrarily long and still have the outcome that the target state has a probability amplitude that is at least an order of magnitude larger than any other state. Does this mean we could use Grover's algorithm to reliably find the target state?

</summary>

__Answer:__

No. Suppose we repeated the first activity with 20 qubits, and we run the quantum circuit a number of times `num_shots = 10,000`. A uniform probability distribution would mean that every state has a probability of $10,000/2^{20}=0.00954$ of being measured even a single time. If the probability of measuring the target state were 10 times that of non-solutions (and the probability of each non-solution were correspondingly slightly decreased), there would only be about a 10% chance of measuring the target state even once. It would be highly unlikely to measure the target state multiple times, which would make it indistinguishable from the many randomly-obtained non-solution states. The good news is that we can obtain even higher-fidelity results by using error suppression and mitigation.

</details>


# Activity 2: An accurate query algorithm workflow

We will start this activity exactly as the first one, except that now you will pair up with another Qiskit enthusiast. You will pick a secret bitstring, and your partner will pick a (generally) different bitstring. You will each generate a quantum circuit that functions as an oracle, and you will exchange them. You will then use Grover's algorithm with that oracle to determine your partner's secret bitstring.

## Step 1: Map classical inputs to a quantum problem

Using the `grover_oracle` function defined above, construct an oracle circuit for one or more marked states. Make sure you tell your partner how many states you have marked, so they can apply the Grover operator the optimal number of times. **Don't make your bitstring too long. 3-5 bits should work without much difficulty.** Longer bitstrings would result in deep circuits that require more advanced techniques like error mitigation.

In [173]:
# Modify the marked states to mark those you wish to target.
marked_states = ["1000"]
oracle = grover_oracle(marked_states)

Dies sind tatsächlich ziemlich große Zahlen, selbst für diesen einfachen Fall. Da alle Quantengates (und insbesondere Zwei-Qubit-Gates) Fehler erfahren und Rauschen unterliegen, würde eine Serie von über 100 Zwei-Qubit-Gates nichts als Rauschen ergeben, wenn die Qubits nicht extrem leistungsfähig wären. Schauen wir uns an, wie diese funktionieren.

## Mit Qiskit-Primitiven ausführen
Wir möchten viele Messungen durchführen und sehen, welcher Zustand am wahrscheinlichsten ist. Eine solche Amplituden-Amplifikation ist ein Sampling-Problem, das für die Ausführung mit dem `Sampler` Qiskit Runtime-Primitiv geeignet ist.

Beachten Sie, dass die `run()`-Methode von Qiskit Runtime SamplerV2 ein Iterable von Primitive Unified Blocks (PUBs) annimmt. Für Sampler ist jedes PUB ein Iterable im Format (circuit, parameter_values). Mindestens benötigt es jedoch eine Liste von Quantenschaltkreis(en).

In [ ]:
from qiskit import qpy

# Save to a QPY file at a location where you can easily find it.
# You might want to specify a global address.
with open("C:\\Users\\...put your own address here...\\my_circuit.qpy", "wb") as f:
    qpy.dump(oracle, f)

Um das Beste aus dieser Erfahrung herauszuholen, empfehlen wir dringend, dass Sie Ihre Experimente auf den von IBM Quantum verfügbaren echten Quantencomputern durchführen. Wenn Sie jedoch Ihre QPU-Zeit erschöpft haben, können Sie die folgenden Zeilen auskommentieren, um diese Aktivität mit einem Simulator abzuschließen.

In [ ]:
from qiskit import qpy

# Load the circuit from your partner's qpy file from the folder where you saved it.
with open("C:\\Users\\...file location here...\\my_circuit.qpy", "rb") as f:
    circuits = qpy.load(f)

# qpy.load always returns a list of circuits
oracle_partner = circuits[0]

# You could visualize the circuit, but this would break the model of a query algorithm.
# oracle_partner.draw("mpl")

## Schritt 4: Nachbearbeitung und Rückgabe des Ergebnisses im gewünschten klassischen Format
Jetzt können wir die Ergebnisse unseres Samplings in einem Histogramm darstellen.

In [174]:
# Update according to your partner's number of target states.
num_marked_states = 1

![Ausgabe der vorherigen Code-Zelle](../../../learning/images/modules/computer-science/grovers/extracted-outputs/96a9107e-0.avif)

Wir sehen, dass der Grover-Algorithmus den gewünschten Zustand mit der höchsten Wahrscheinlichkeit zurückgab, mindestens eine Größenordnung höher als andere Optionen. In der nächsten Aktivität werden wir den Algorithmus auf eine Weise verwenden, die konsistenter mit dem Zwei-Parteien-Workflow eines Abfragealgorithmus ist.

### Überprüfen Sie Ihr Verständnis
Lesen Sie die Fragen unten, denken Sie über Ihre Antwort nach, dann klicken Sie auf das Dreieck, um die Lösung zu enthüllen.

<details>
<summary>

Wir haben gerade nach einer einzelnen Lösung in einer Menge von $2^4=16$ möglichen Zuständen gesucht. Wir haben die optimale Anzahl von Wiederholungen des Grover-Operators als $t=3$ bestimmt. Würde diese optimale Zahl zunehmen oder abnehmen, wenn wir nach (a) einer von mehreren Lösungen suchen würden, oder (b) einer einzelnen Lösung in einem Raum von mehr möglichen Zuständen?

</summary>

__Antwort:__

Erinnern Sie sich daran, dass wir, solange die Anzahl der Lösungen im Vergleich zum gesamten Lösungsraum klein ist, die Sinusfunktion um kleine Winkel erweitern und verwenden können
$$
(2t+1)\theta = (2t+1) \sin^{-1}{\sqrt{\frac{|\mathcal{A}_1|}{N}}}\approx (2t+1) \sqrt{\frac{|\mathcal{A}_1|}{N}} \approx \pi/2\\

t \approx \frac{\pi}{4}\sqrt{\frac{N}{|\mathcal{A}_1|}}-\frac{1}{2}
$$

(a) Wir sehen aus dem obigen Ausdruck, dass die Erhöhung der Anzahl von Lösungszuständen die Anzahl der Iterationen verringern würde. Vorausgesetzt, dass der Bruch $\frac{|\mathcal{A}_1|}{N}$ noch klein ist, können wir beschreiben, wie $t$ abnehmen würde: $t~\frac{1}{\sqrt{|\mathcal{A}_1|}}.$

(b) Wenn der Raum möglicher Lösungen ($N$) zunimmt, erhöht sich die Anzahl der erforderlichen Iterationen, aber nur wie $t~\sqrt{N}$.

</details>

<details>
<summary>

Angenommen, wir könnten die Größe des Ziel-Bitstrings beliebig lang erhöhen und hätten dennoch das Ergebnis, dass der Zielzustand eine Wahrscheinlichkeitsamplitude hat, die mindestens eine Größenordnung größer ist als jeder andere Zustand. Bedeutet dies, dass wir den Grover-Algorithmus verwenden könnten, um zuverlässig den Zielzustand zu finden?

</summary>

__Antwort:__

Nein. Angenommen, wir würden die erste Aktivität mit 20 Qubits wiederholen und den Quantenschaltkreis eine Anzahl von Malen `num_shots = 10.000` ausführen. Eine gleichmäßige Wahrscheinlichkeitsverteilung würde bedeuten, dass jeder Zustand eine Wahrscheinlichkeit von $10.000/2^{20}=0,00954$ hat, auch nur ein einziges Mal gemessen zu werden. Wenn die Wahrscheinlichkeit, den Zielzustand zu messen, 10 Mal so hoch wäre wie die von Nicht-Lösungen (und die Wahrscheinlichkeit jeder Nicht-Lösung entsprechend leicht verringert würde), gäbe es nur etwa eine 10%-ige Chance, den Zielzustand auch nur einmal zu messen. Es wäre sehr unwahrscheinlich, den Zielzustand mehrmals zu messen, was ihn von den vielen zufällig erhaltenen Nicht-Lösungszuständen nicht zu unterscheiden machen würde. Die gute Nachricht ist, dass wir noch höhere Treue-Ergebnisse durch Verwendung von Fehlerunterdrückung und -minderung erhalten können.

</details>

# Aktivität 2: Ein präziser Abfragealgorithmus-Workflow
Wir beginnen diese Aktivität genau wie die erste, außer dass Sie sich jetzt mit einem anderen Qiskit-Enthusiasten zusammentun werden. Sie werden einen geheimen Bitstring wählen, und Ihr Partner wird einen (im Allgemeinen) anderen Bitstring wählen. Sie werden jeweils einen Quantenschaltkreis generieren, der als Orakel fungiert, und Sie werden sie austauschen. Sie werden dann den Grover-Algorithmus mit diesem Orakel verwenden, um den geheimen Bitstring Ihres Partners zu bestimmen.

## Schritt 1: Klassische Eingaben auf ein Quantenproblem abbilden
Konstruieren Sie mit der oben definierten `grover_oracle`-Funktion einen Orakel-Schaltkreis für einen oder mehrere markierte Zustände. Stellen Sie sicher, dass Sie Ihrem Partner mitteilen, wie viele Zustände Sie markiert haben, damit er den Grover-Operator die optimale Anzahl von Malen anwenden kann. **Machen Sie Ihren Bitstring nicht zu lang. 3-5 Bits sollten ohne große Schwierigkeiten funktionieren.** Längere Bitstrings würden zu tiefen Schaltkreisen führen, die fortgeschrittenere Techniken wie Fehlerminderung erfordern.

In [175]:
grover_op = grover_operator(oracle_partner)
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(num_marked_states / 2**grover_op.num_qubits)))
)
qc = QuantumCircuit(grover_op.num_qubits)
qc.h(range(grover_op.num_qubits))
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
qc.measure_all()

Jetzt haben Sie einen Quantenschaltkreis erstellt, der die Phase Ihres Zielzustands umdreht. Sie können diesen Schaltkreis als `my_circuit.qpy` mit der unten stehenden Syntax speichern.

In [176]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
backend.name

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
circuit_partner_isa = pm.run(qc)

Senden Sie diese Datei jetzt an Ihren Partner (per E-Mail, Messaging-Service, einem gemeinsamen Repo usw.). Lassen Sie Ihren Partner auch seinen Schaltkreis an Sie senden. Stellen Sie sicher, dass Sie die Datei an einem Ort speichern, wo Sie sie leicht finden können. Sobald Sie den Schaltkreis Ihres Partners haben, könnten Sie ihn visualisieren - aber das würde das Abfragemodell brechen. Das heißt, wir modellieren eine Situation, in der Sie das Orakel abfragen können (den Orakel-Schaltkreis verwenden), es aber nicht untersuchen können, um zu bestimmen, welchen Zustand es anvisiert.

In [ ]:
# To run on a real quantum computer (this was tested on a Heron r2 processor and used 4 seconds of QPU time)

from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(mode=backend)
sampler.options.default_shots = 10_000
result = sampler.run([circuit_partner_isa]).result()
dist = result[0].data.meas.get_counts()

Fragen Sie Ihren Partner, wie viele Zielzustände er codiert hat, und geben Sie es unten ein.

In [114]:
plot_distribution(dist)

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/ee7a59ac-0.avif" alt="Output of the previous code cell" />

Dies wird im nächsten Ausdruck verwendet, um die optimale Anzahl von Grover-Iterationen zu bestimmen.

In [ ]:
from qiskit import QuantumCircuit


def grover_oracle_hamming_weight(num_qubits, target_weight):
    """
    Build a Grover oracle that marks all states with Hamming weight == target_weight.

    Parameters:
        num_qubits (int): Number of qubits in the circuit.
        target_weight (int): The Hamming weight to mark.

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle.
    """
    qc = QuantumCircuit(num_qubits)
    marked_count = 0
    marked_list = []
    for x in range(2**num_qubits):
        bitstr = format(x, f"0{num_qubits}b")
        if bitstr.count("1") == target_weight:
            # Count the number of target states
            marked_count = marked_count + 1
            marked_list.append(bitstr)
            # Reverse for Qiskit bit order (qubit 0 is rightmost)
            rev_target = bitstr[::-1]
            zero_inds = [i for i, b in enumerate(rev_target) if b == "0"]
            # Apply X gates to 'open' controls (where bit is 0)
            qc.x(zero_inds)
            # Multi-controlled Z (as MCX conjugated by H)
            if num_qubits == 1:
                qc.z(0)
            else:
                qc.h(num_qubits - 1)
                qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)
                qc.h(num_qubits - 1)
            # Undo X gates
            qc.x(zero_inds)
    return qc, marked_count, marked_list

In [20]:
# Completing step 1
oracle, num_marked_states, marked_states = grover_oracle_hamming_weight(3, 2)
print(marked_states)
oracle.draw(output="mpl", style="iqp")

['011', '101', '110']


<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/5f5e4675-1.avif" alt="Output of the previous code cell" />

## Schritt 3: Mit Qiskit-Primitiven ausführen
Dies ist ebenfalls identisch mit dem Prozess in der ersten Aktivität.

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Completing step 1
oracle, num_marked_states, marked_states = grover_oracle_hamming_weight(4, 2)
oracle.draw(output="mpl", style="iqp")

grover_op = grover_operator(oracle)
grover_op.decompose().draw(output="mpl", style="iqp")
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)

qc = QuantumCircuit(grover_op.num_qubits)
qc.h(range(grover_op.num_qubits))
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
qc.measure_all()
qc.draw(output="mpl", style="iqp")

# Step 2: Optimize for running on real quantum computers

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(backend.name)

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
circuit_isa = pm.run(qc)

# Step 3: Execute using Qiskit primitives
# To run on a real quantum computer (this was tested on a Heron r2 processor and used 4 seconds of QPU time)

sampler = Sampler(mode=backend)
sampler.options.default_shots = 10_000
result = sampler.run([circuit_isa]).result()
dist = result[0].data.meas.get_counts()

# To run on local simulator:
# from qiskit.primitives import StatevectorSampler as Sampler
# sampler = Sampler()
# result = sampler.run([qc]).result()
# dist = result[0].data.meas.get_counts()

qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-08-08 14:14:51,686: Default instance not set. Searching all available instances.


ibm_brisbane


In [ ]:
print("The total depth is ", circuit_isa.depth())
print(
    "The depth of two-qubit gates is ",
    circuit_isa.depth(lambda instruction: instruction.operation.num_qubits == 2),
)

The total depth is  502
The depth of two-qubit gates is  130


In [ ]:
num_marked_states

6

In [ ]:
plot_distribution(dist)

<Image src="../../../learning/images/modules/computer-science/grovers/extracted-outputs/cf8b6be8-0.avif" alt="Output of the previous code cell" />

Here Grover's algorithm did indeed prepare the states with Hamming weight 2 to be much more likely to be measured than any other states, roughly one order of magnitude more likely.

### Critical concepts:

In this module, we learned some key features of Grover's algorithm:
- Whereas classical unstructured search algorithms require a number of queries that scales linearly in the size of the space, $N,$ Grover's algorithm requires a number of queries that scales like $\sqrt{N}.$
- Grover's algorithm involves repeating a series of operations (commonly called the "Grover operator") a number of times $t,$ chosen to make the target states optimally likely to be measured.
- Grover's algorithm can be run with fewer than $t$ iterations and still amplify the target states.
- Grover's algorithm fits into the query model of computation and makes the most sense when one person controls the search and another controls/constructs the oracle. It may also be useful as a subroutine in other quantum computations.

## Questions

### T/F questions:

1. T/F Grover's algorithm provides an exponential improvement over classical algorithms in the number of queries needed to find a single marked state in unstructured search.

2. T/F Grover's algorithm works by iteratively increasing the probability that a solution state will be measured.

3. T/F The more times you iterate the Grover operator, the higher the probability of measuring a solution state.

### MC questions:

1. Select the best option to complete the sentence. The best strategy to successfully use Grover's algorithm on modern quantum computers is to iterate the Grover operator...
- a. Only once.
- b. Always $t$ times, to maximize the solution state(s)' probability amplitude.
- c. Up to $t$ times, though fewer may be enough to make solution states stand out.
- d. No fewer than 10 times.


2. A phase query circuit is shown here that functions as an oracle to mark a certain state with a phase flip. Which of the following states get marked by this circuit?

![An image of a simple grover oracle.](../../../learning/images/modules/computer-science/grovers/grover-oracle-question.avif)

- a. $|0000\rangle$
- b. $|0101\rangle$
- c. $|0110\rangle$
- d. $|1001\rangle$
- e. $|1010\rangle$
- f. $|1111\rangle$


3. Suppose you want to search for three marked states from a set of 128. What is the optimal number of iterations of the Grover operator to maximize the amplitudes of the marked states?
- a. 1
- b. 3
- c. 5
- d. 6
- e. 20
- f. 33


### Discussion questions:

1. What other conditions might you use in a quantum oracle? Consider conditions that are easy to state or impose on a bitstring but are not merely writing out the marked bitstrings.


2. Can you see any problems with scaling Grover's algorithm on modern quantum computers?